In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def import_24bit_data(fn):
    table = pd.read_csv(fn, sep=',', skiprows = 0)
    table.columns = np.arange(0,523,1)
    
    data_ch1 = []
    for i in range(len(table[0])):
        for j in range(40):
            data_ch1.append(table[j][i])
            
    data_ch2 = []
    for i in range(len(table[40])):
        for j in range(40):
            data_ch2.append(table[40+j][i])
            
    data_ch3 = []
    for i in range(len(table[80])):
        for j in range(40):
            data_ch3.append(table[80+j][i])
            
    data_ch4 = []
    for i in range(len(table[120])):
        for j in range(40):
            data_ch4.append(table[120+j][i])
            
    data_ch5 = []
    for i in range(len(table[160])):
        for j in range(40):
            data_ch5.append(table[160+j][i])
            
    data_ch6 = []
    for i in range(len(table[200])):
        for j in range(40):
            data_ch6.append(table[200+j][i])
            
    clock_ticks = []
    for i in range(len(table[240+240])):
        for j in range(40):
            clock_ticks.append(table[240+240+j][i])
    
    data_ch1 = np.array(data_ch1)
    data_ch2 = np.array(data_ch2)
    data_ch3 = np.array(data_ch3)
    data_ch4 = np.array(data_ch4)
    data_ch5 = np.array(data_ch5)
    data_ch6 = np.array(data_ch6)
    clock_ticks = np.array(clock_ticks)
    packet_numbers = np.array(table[520])
    times_1 = np.array(table[521])
    times_2 = np.array(table[522])
    
    clock_ticks_diff = [clock_ticks[i+1] - clock_ticks[i] for i in range(len(clock_ticks)-1)]
    print("NUMBER OF DATA POINTS:", len(data_ch1))
    print("MEAN CLOCK TICK DIFF:", np.mean(clock_ticks_diff))
    print("MEAN PERIOD:", np.mean(clock_ticks_diff)/1e6)
    print("MEAN FREQ (seconds):", 1/(np.mean(clock_ticks_diff)/1e6))
        
    return data_ch1, data_ch2, data_ch3, data_ch4, data_ch5, data_ch6, clock_ticks, packet_numbers, times_1, times_2, 1/(np.mean(clock_ticks_diff)/1e6)

In [3]:
def shuyu_fft(data, bin_size, freq):
    """
    data = data
    bin_size = integer number of data points to bin and FFT together
    freq = sampling freq
    """
    
    num_bins = int(len(data)/bin_size)
        
    fft_average = np.zeros(bin_size // 2)
    fft_ = np.zeros((num_bins, bin_size // 2))
    
    fft_freq = np.fft.fftfreq(bin_size, d=1/freq)[:bin_size // 2]
    
    for i in range(num_bins):
        data_bin = data[i*bin_size : (i + 1)*bin_size]
        
        fft_bin = np.fft.fft(data_bin)[:bin_size // 2]
        
        fft_[i] = (2 / bin_size) * np.abs(fft_bin)
        
        fft_average += fft_[i]
    
    fft_average = fft_average / num_bins
    
    return fft_, fft_average, fft_freq, num_bins 

In [5]:
#plot fft of each channel

def shuyu_fft_multi_1(datasets, freq, bin_size_in_seconds, title): 
    """
    datasets = 6 arrays with voltage data
    bin_size = integer number of seconds to make each bin to FFT
    freq = sampling frequency of ADC
    title = title string
    """
    
    if len(datasets) != 6:
        raise ValueError("This function requires exactly 6 datasets.")
        
    plt.figure(figsize=(18, 12))

    for i, dataset in enumerate(datasets):
        
        fft_, fft_average, fft_freq, num_bins = shuyu_fft(dataset, int(freq*bin_size_in_seconds), freq)
        valid_indices = fft_freq > 0

        # Create a subplot for each dataset
        plt.subplot(3, 2, i + 1)
        for j in range(num_bins):
            plt.plot(fft_freq[valid_indices], fft_[j][valid_indices], alpha=0.01)

        plt.plot(fft_freq[valid_indices], fft_average[valid_indices], color='black', label='Average')
        plt.xlabel('Frequency (Hz)')
        plt.grid()

        # Add labels and customize each subplot
        plt.title(f"Channel {i + 1}")
        plt.axvline(60, c='cyan', label='60Hz', alpha=0.4)
        plt.axvline(120, c='magenta', label='120Hz', alpha=0.4)
        plt.legend()
        plt.xscale('log')
        plt.yscale('log')
        plt.xlim(1e-1, int(freq / 2))
        plt.ylabel('Acceleration (m/s^2)')

    # Add a title for the entire figure
    plt.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit the title
    plt.show()

In [6]:
#plot acceleration/displacement each accelerometer seperately

def shuyu_fft_multi_2(datasets, freq, bin_size_in_seconds):
    bin_size = int(freq * bin_size_in_seconds)
    
    plt.figure(figsize=(18, 12))
    
    fft_averages = []
    fft_displacements = []
    fft_freq = None
    
    for i,dataset in enumerate(datasets):
        #fft
        fft_, fft_average_, fft_freq_, _ = shuyu_fft(dataset, bin_size, freq)
        fft_averages.append(fft_average_)
        #displacement
        displacement_ = np.zeros_like(fft_average_)
        valid_indices = fft_freq_ > 0  # Avoid division by zero at f=0
        displacement_[valid_indices] = fft_average_[valid_indices] / (2 * np.pi * fft_freq_[valid_indices])**2
        fft_displacements.append(displacement_)
        #freq
        fft_freq = fft_freq_
    
    # Accelerometer 1
    plt.subplot(2, 2, 1)
    plt.plot(fft_freq, fft_averages[2], c='#fdae61', label='a1 x', alpha=0.8)
    plt.plot(fft_freq, fft_averages[0], c='#d7191c', label='a1 y', alpha=0.8)
    plt.plot(fft_freq, fft_averages[1], c='#2b83ba', label='a1 z', alpha=0.8)
    plt.title("Accelerometer 1 (Acceleration)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Acceleration (m/s²)")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    
    # Accelerometer 2
    plt.subplot(2, 2, 2)
    plt.plot(fft_freq, fft_averages[3], c='#fdae61', label='a2 x', alpha=0.8)
    plt.plot(fft_freq, fft_averages[4], c='#d7191c', label='a2 y', alpha=0.8)
    plt.plot(fft_freq, fft_averages[5], c='#2b83ba', label='a2 z', alpha=0.8)
    plt.title("Accelerometer 2 (Acceleration)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Acceleration (m/s²)")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    
    # Accelerometer 1
    plt.subplot(2, 2, 3)
    plt.plot(fft_freq, fft_displacements[2], c='#fdae61', label='a1 x', alpha=0.8)
    plt.plot(fft_freq, fft_displacements[0], c='#d7191c', label='a1 y', alpha=0.8)
    plt.plot(fft_freq, fft_displacements[1], c='#2b83ba', label='a1 z', alpha=0.8)
    plt.title("Accelerometer 1 (Displacement - Amp/w^2)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Displacement (m)")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    
    # Accelerometer 2
    plt.subplot(2, 2, 4)
    plt.plot(fft_freq, fft_displacements[3], c='#fdae61', label='a2 x', alpha=0.8)
    plt.plot(fft_freq, fft_displacements[4], c='#d7191c', label='a2 y', alpha=0.8)
    plt.plot(fft_freq, fft_displacements[5], c='#2b83ba', label='a2 z', alpha=0.8)
    plt.title("Accelerometer 2 (Displacement - Amp/w^2)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Acceleration (m)")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    plt.show()

# Example

In [ ]:
data_ch1, data_ch2, data_ch3, data_ch4, data_ch5, data_ch6, clock_ticks, packet_numbers, times_1, times_2, freq = import_24bit_data('20250213_2.csv')

In [ ]:
def printstuff(caption):
    print("\n" + caption, "\n x1:",np.mean(datasets[2]), "\n y1:", np.mean(datasets[0]),  "\n z1:", np.mean(datasets[1]))
    print(" x2:", np.mean(datasets[3]), "\n y2:", np.mean(datasets[4]), "\n z2:", np.mean(datasets[5]))

datasets = np.array([data_ch1, data_ch2, data_ch3, data_ch4, data_ch5, data_ch6])
printstuff("raw data (adc counts)")

datasets = 1.25 + datasets*(2.5/(2**24-1))
printstuff("voltage (V):")

datasets = (datasets - 2.5)*(2/1.1)
printstuff("acceleration (g):")

print("\n std (g):", "\n x1:",np.std(datasets[2]), "\n y1:", np.std(datasets[0]),  "\n z1:", np.std(datasets[1]))
print(" x2:", np.std(datasets[3]), "\n y2:", np.std(datasets[4]), "\n z2:", np.std(datasets[5]))

datasets = datasets*(-9.81)
printstuff("acceleration (m/s²):")

print("\n std (m/s²):", "\n x1:",np.std(datasets[2]), "\n y1:", np.std(datasets[0]),  "\n z1:", np.std(datasets[1]))
print(" x2:", np.std(datasets[3]), "\n y2:", np.std(datasets[4]), "\n z2:", np.std(datasets[5]))

print("\n Actual Frequency (Hz):", 8074/6) #Sampling rate set on the ADC board
print("\n Microcontroller's opinion of Frequency (Hz):", freq) #What the microcontroller thinks the sampling rate is (spoiler: it's stupid)

In [ ]:
shuyu_fft_multi_2(datasets, 8074/6, 10)